In [1]:
import tensorflow as tf
import os

In [2]:
# get current dir
current_dir=os.path.dirname(os.path.realpath(__name__))

In [3]:
# dataset url
#
#the next command will create a dir called 'content' in your current location - be aware
#

_url='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

#download command
zip_file=tf.keras.utils.get_file(origin=_url,fname='flower_photos.tgz',extract=True,cache_subdir=current_dir+'/content')

#add dir to path
base_dir=os.path.join(os.path.dirname(zip_file),'flower_photos')

228818944/228813984 [==============================] - 9s 0us/step


In [4]:
#not needed anymore
del current_dir

In [5]:
_IMAGE_SIZE=224
_BATCH_SIZE=64


In [6]:
# validation_split - split to 20% test & 80% train
data_gen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [7]:
# splitted to training and validation
# 80% train =>

train_gen=data_gen.flow_from_directory(
    base_dir,
target_size=(_IMAGE_SIZE,_IMAGE_SIZE),
batch_size=_BATCH_SIZE,
subset='training'
)

Found 2939 images belonging to 5 classes.


In [8]:
# 20% validation =>

val_gen=data_gen.flow_from_directory(
    base_dir,
target_size=(_IMAGE_SIZE,_IMAGE_SIZE),
batch_size=_BATCH_SIZE,
subset='validation'
)

Found 731 images belonging to 5 classes.


In [9]:
lables='\n'.join(sorted(train_gen.class_indices.keys()))
with open('labels.txt','w') as f:
    f.write(lables)

#del from memory
del lables

In [10]:
_IMG_SHAPE=(_IMAGE_SIZE,_IMAGE_SIZE,3)
base_model=tf.keras.applications.MobileNetV2(
    input_shape=_IMG_SHAPE,
    include_top=False,
    weights='imagenet'
    )

9412608/9406464 [==============================] - 1s 0us/step


In [11]:
base_model.trainable=False

In [12]:
#short import to make the next command shorter
from tensorflow.keras import layers

# 5 = num of classes
model=tf.keras.Sequential([
    base_model,
layers.Conv2D(32,3,activation='relu'),
layers.Dropout(0.2),
layers.GlobalAveragePooling2D(),
layers.Dense(5,activation='softmax')
])

In [13]:
model.compile(
optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',
metrics=['accuracy']
 )

In [14]:
import PIL

In [15]:
#fitting the model
epochs=10
history=model.fit(
    train_gen,
    epochs=epochs,
    validation_data=val_gen
    )

Epoch 1/10
46/46 [==============================] - 31s 664ms/step - loss: 1.2887 - accuracy: 0.5723 - val_loss: 0.7494 - val_accuracy: 0.7127
Epoch 2/10
46/46 [==============================] - 30s 659ms/step - loss: 0.6337 - accuracy: 0.7173 - val_loss: 0.6408 - val_accuracy: 0.6867
Epoch 3/10
46/46 [==============================] - 30s 659ms/step - loss: 0.3526 - accuracy: 0.8486 - val_loss: 0.4079 - val_accuracy: 0.8687
Epoch 4/10
46/46 [==============================] - 30s 655ms/step - loss: 0.1776 - accuracy: 0.9411 - val_loss: 0.4318 - val_accuracy: 0.8468
Epoch 5/10
46/46 [==============================] - 30s 649ms/step - loss: 0.1157 - accuracy: 0.9622 - val_loss: 0.4810 - val_accuracy: 0.8495
Epoch 6/10
46/46 [==============================] - 30s 657ms/step - loss: 0.0778 - accuracy: 0.9789 - val_loss: 0.4673 - val_accuracy: 0.8700
Epoch 7/10
46/46 [==============================] - 30s 651ms/step - loss: 0.0639 - accuracy: 0.9826 - val_loss: 0.5087 - val_accuracy: 0.8618

In [16]:
saved_model_dir=''
tf.saved_model.save(model,saved_model_dir)
converter=tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model=converter.convert()

with open('model.tflite','wb') as f:
    f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: assets


In [ ]:
# in the project dir there will be many new files 
# we want only the labels.txt and model.tflite files.

In [ ]:
# in the video he used commands to download from the colab website
# i've done it localy, so just copy to the assets folder in the flutter projects.